In [2]:
pip install langchain

In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [4]:
pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [5]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

··········


In [20]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Mahipal")])

AIMessage(content='Hello Mahipal! Nice to meet you! Is there something I can help you with or would you like to chat?', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 17, 'total_tokens': 43, 'completion_time': 0.020112164, 'prompt_time': 0.003229595, 'queue_time': None, 'total_time': 0.023341758999999997}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-179ca22f-0f1b-46dd-a22d-3ead514bd03f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 26, 'total_tokens': 43})

In [21]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know or recall personal information about individuals, including their names. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations.\n\nIf you'd like to share your name with me, I'm happy to learn it and address you by name in our conversation.", response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 15, 'total_tokens': 99, 'completion_time': 0.066148036, 'prompt_time': 0.00296762, 'queue_time': None, 'total_time': 0.069115656}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_33d61fdfc3', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1653169-4157-418f-a170-63c82b469489-0', usage_metadata={'input_tokens': 15, 'output_tokens': 84, 'total_tokens': 99})

In [22]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Mahipal"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You told me your name is Mahipal!', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 42, 'total_tokens': 53, 'completion_time': 0.008020938, 'prompt_time': 0.007174059, 'queue_time': None, 'total_time': 0.015194997}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_33d61fdfc3', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1fd71f4-65b5-4b1a-8345-f2343f9d1db6-0', usage_metadata={'input_tokens': 42, 'output_tokens': 11, 'total_tokens': 53})

In [10]:
# ! pip install langchain_community

In [23]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [12]:
config = {"configurable": {"session_id": "abc2"}}

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Mahipal")],
    config=config,
)

response.content

'Nice to meet you, Mahipal! How are you doing today?'

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Mahipal!'

In [26]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I apologize, but I don't know your name. I'm a large language model, I don't have the ability to retain information about individual users or recall previous conversations. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. Would you like to introduce yourself?"

In [27]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is still Mahipal!'

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Mahipal")]})

response.content

"Hi Mahipal! It's nice to meet you! I'm here to help with any questions or topics you'd like to discuss. What's on your mind today?"

In [31]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [32]:
config = {"configurable": {"session_id": "abc5"}}

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Amit")],
    config=config,
)

response.content

"Hi Amit! It's great to meet you too! I'm here to assist you with any questions or topics you'd like to discuss. What's on your mind today?"

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I think I got tricked earlier! According to our conversation, your name is Jim. Am I right?'

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [38]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Mahipal")], "language": "Spanish"}
)

response.content

'Hola Mahipal! Me alegra conocerte. Estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?'

In [39]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [40]:
config = {"configurable": {"session_id": "abc11"}}

In [41]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm krishna")], "language": "Spanish"},
    config=config,
)

response.content

'Hola Krishna! ¡Encantado de conocerte! (Hello Krishna! Nice to meet you!) ¿En qué puedo ayudarte hoy? (What can I help you with today?)'

In [42]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'¡Claro! Tu nombre es Krishna! (Of course! Your name is Krishna!)'

In [43]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Mahipal"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm Mahipal"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'Mahipal!'

In [45]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked the math problem "whats 2 + 2"'

In [46]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [47]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Mahipal!'

In [49]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm Mahipal. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Mah|ip|al|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| laugh|!| Do| you| want| to| hear| another| one|?||